In [ ]:
!pip install torch transformers datasets accelerate peft bitsandbytes mauve-text


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/teamspace/studios/this_studio/dataset_train.json")

In [ ]:
from huggingface_hub import login

#login()


In [ ]:
from transformers import AutoTokenizer

# Carica il tokenizer pre-addestrato del modello Llama 2 7B in formato Hugging Face.
# Il tokenizer serve a convertire testo in token numerici comprensibili dal modello.
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

# Imposta il token di padding uguale al token di fine sequenza (EOS token).
# Questo è utile perché alcuni modelli non hanno un token di padding dedicato.
tokenizer.pad_token = tokenizer.eos_token


# Funzione per tokenizzare un esempio del dataset
def tokenize(example):
    # Crea un prompt strutturato a partire dai campi del dataset
    # 'instruction', 'input', 'output'. Il prompt ha la forma:
    # ### Instruction:
    # <istruzione>
    # ### Input:
    # <input>
    # ### Response:
    # <output>
    # Questo formato è spesso usato per addestrare modelli instruction-following.
    prompt = f"### Instruction:\n{example['instruction']}\n### Input:\n{example['input']}\n### Response:\n{example['output']}"
    
    # Tokenizza il prompt:
    # - truncation=True → tronca il testo se supera max_length
    # - padding="max_length" → aggiunge padding fino a max_length
    # - max_length=1024 → lunghezza massima dei token
    # Restituisce un dizionario con input_ids e attention_mask pronto per il modello.
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=1024)


# Applica la funzione di tokenizzazione a tutto il dataset.
# `dataset.map()` crea un nuovo dataset in cui ogni esempio è già tokenizzato.
tokenized_dataset = dataset.map(tokenize)


In [ ]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

# Carica il modello Llama-2 7B pre-addestrato in modalità Causal Language Modeling.
# load_in_8bit=True → utilizza la quantizzazione a 8 bit per ridurre l'uso di memoria.
# device_map="auto" → assegna automaticamente i layer del modello ai dispositivi disponibili (CPU/GPU).
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    load_in_8bit=True,
    device_map="auto"
)

# Configurazione per il LoRA (Low-Rank Adaptation)
# LoRA permette di fare fine-tuning aggiungendo pochi parametri senza aggiornare tutto il modello.
peft_config = LoraConfig(
    r=8,                        # Rank della matrice di aggiornamento low-rank
    lora_alpha=32,               # Moltiplicatore di scaling per stabilizzare l’addestramento LoRA
    target_modules=["q_proj", "v_proj"],  # Layer del modello dove applicare LoRA (tipicamente Q e V delle attention)
    lora_dropout=0.05,           # Dropout applicato ai pesi LoRA per regolarizzazione
    bias="none",                 # Non aggiunge bias aggiuntivo nel fine-tuning
    task_type=TaskType.CAUSAL_LM # Tipo di task: Causal Language Modeling
)

# Applica LoRA al modello originale.
# Restituisce un modello PEFT che contiene i pesi originali congelati + i parametri LoRA addestrabili.
model = get_peft_model(model, peft_config)


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Configurazione dei parametri di training
training_args = TrainingArguments(
    output_dir="./llama-finetuned-generation",  # Cartella dove salvare i checkpoint del modello fine-tuned
    per_device_train_batch_size=2,    # Dimensione del batch per GPU/TPU/CPU
    gradient_accumulation_steps=4,    # Accumula i gradienti per simulare batch più grandi
    num_train_epochs=1,               # Numero di epoche di training sul dataset
    learning_rate=2e-4,               # Learning rate per l'ottimizzatore
    logging_dir="./logs-generation",             # Cartella per salvare i log di training
    logging_steps=10,                 # Frequenza (in step) di scrittura dei log
    save_strategy="epoch",            # Salva il modello alla fine di ogni epoca
    fp16=True                         # Abilita mixed precision (half precision) per ridurre uso memoria e velocizzare training
)

# Collator dei dati per il Language Modeling
# Prepara batch di input per il modello
# mlm=False → modello non usa masked language modeling, adatto a causal LM come Llama
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Creazione del Trainer di Hugging Face
trainer = Trainer(
    model=model,                      # Modello da allenare
    args=training_args,                # Parametri di training definiti sopra
    train_dataset=tokenized_dataset["train"],  # Dataset di training già tokenizzato
    tokenizer=tokenizer,               # Tokenizer associato al modello
    data_collator=data_collator        # Funzione che prepara i batch durante il training
)

# Avvia il training del modello
trainer.train()


In [ ]:
prompt = """### Instruction:
You are an expert in generating phishing emails. Your purpose is to generate realistic phishing emails by using information taken from the input you receive from the user.
### Input:
Generate an email that impersonates a pharmacy or online healthcare service, specifically targeting users who have placed recent orders. Craft the message to appear as a genuine refill reminder, created by an automated system to assist customers in receiving their necessary supplies quickly. The tone should be helpful and courteous. Include a clear call-to-action for the recipient to click on a provided link to use the refill system. Infuse the email with a sense of routine customer service without raising suspicion. Ensure the closing includes a signature from a seemingly legitimate representative. Avoid overtly technical or suspicious language, aiming instead for clarity and ease of use.
### Response:
"""

enc = tokenizer(                                               # Tokenizza il prompt usando il tokenizer del modello
    prompt,                                                    # Testo da tokenizzare
    return_tensors="pt",                                       # Richiede tensori PyTorch come output
    padding=True,                                              # Applica padding automatico
)

input_ids = enc.input_ids.cuda()                               # Sposta gli input IDs sulla GPU
attention_mask = enc.attention_mask.cuda()                     # Sposta la attention mask sulla GPU

outputs = model.generate(                                      # Genera l’output del modello
    input_ids=input_ids,                                       # Fornisce i token di input
    attention_mask=attention_mask,                             # Fornisce la maschera di attenzione
    max_new_tokens=1024,                                       
    pad_token_id=tokenizer.eos_token_id                        # Specifica il token di padding
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))  # Decodifica e stampa l’output finale del modello

In [ ]:
model.save_pretrained("./llama-finetuned-generation")
tokenizer.save_pretrained("./llama-finetuned-generation")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# ----------------------------
# CONFIGURAZIONE PATH E MODELLO
# ----------------------------

BASE_MODEL_NAME = "meta-llama/Llama-2-7b-hf"
LORA_ADAPTER_PATH = "./llama-finetuned-generation"

# ----------------------------
# CARICAMENTO TOKENIZER
# ----------------------------

tokenizer = AutoTokenizer.from_pretrained(LORA_ADAPTER_PATH)

# Per LLaMA è buona pratica assicurarsi che il pad_token sia definito
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ----------------------------
# CARICAMENTO MODELLO BASE
# ----------------------------

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    load_in_8bit=True,        # oppure load_in_4bit=True se usi QLoRA
    device_map="auto"
)

# ----------------------------
# CARICAMENTO ADAPTER LoRA
# ----------------------------

model = PeftModel.from_pretrained(
    base_model,
    LORA_ADAPTER_PATH
)

model.eval()



In [ ]:
def dataset_metrics(df, name="Dataset"):          # Definisce una funzione che calcola e stampa metriche descrittive di un DataFrame
    print(f"\n📊 Metriche - {name}")               # Stampa il titolo delle metriche, includendo il nome del dataset
    print("-" * 50)                               # Stampa una linea separatrice lunga 50 caratteri
    print(f"Numero righe: {df.shape[0]}")          # Stampa il numero di righe del DataFrame
    print(f"Numero colonne: {df.shape[1]}")       # Stampa il numero di colonne del DataFrame
    print("\nColonne:")                            # Stampa l’intestazione della sezione colonne
    print(df.columns.tolist())                    # Stampa la lista dei nomi delle colonne
    print("\nValori mancanti per colonna:")       # Stampa l’intestazione della sezione sui valori mancanti
    print(df.isnull().sum())                      # Calcola e stampa il numero di valori nulli per ciascuna colonna
    print("\nTipi di dato:")                      # Stampa l’intestazione della sezione sui tipi di dato
    print(df.dtypes)                              # Stampa il tipo di dato associato a ogni colonna


In [ ]:
import pandas as pd                                  # Importa la libreria pandas per la gestione e l’analisi dei dati
import csv                                           # Importa il modulo csv (utile per operazioni su file CSV)
import sys                                           # Importa il modulo sys per interazioni con il sistema (argomenti, I/O, ecc.)

df = pd.read_json("dataset_test.json")               # Carica il file JSON in un DataFrame pandas

dataset_metrics(df, "Dataset di test")  # Chiama la funzione dataset_metrics per stampare le metriche del dataset


In [ ]:
import mauve
import torch

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

p_list = []
q_list = []

dim = len(df)                                  # Numero totale di righe del DataFrame

for i in range(0, dim):                        # Cicla su tutte le righe del DataFrame
    first_row = df.iloc[i]                     # Estrae la riga i-esima come Series

    # Estrai prompt e response separatamente
    user_prompt = first_row['input']            
    response = first_row['output']                 

    general_prompt = f"""### Instruction:
You are an expert in generating phishing emails. Your purpose is to generate realistic phishing emails by using information taken from the input you receive from the user.
### Input:
{user_prompt}
### Response:
"""
    
    enc = tokenizer(                           # Tokenizza il prompt
        general_prompt,
        return_tensors="pt",                   # Restituisce tensori PyTorch
        padding=True,                          # Applica padding automatico
    )

    num_tokens = enc.input_ids.shape[1]        # Calcola il numero totale di token del prompt

    if num_tokens > 1024:                       # Controlla il limite massimo di token consentiti
        # Salta questa riga
        continue                               # Esclude l’email troppo lunga dalla valutazione

    input_ids = enc.input_ids.cuda()           # Sposta gli input IDs sulla GPU
    attention_mask = enc.attention_mask.cuda() # Sposta la attention mask sulla GPU

    outputs = model.generate(                  # Genera la risposta del modello
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=1024,                   
        pad_token_id=tokenizer.eos_token_id    # Specifica il token di padding
    )
    
    risposta = tokenizer.decode(               # Decodifica l’output del modello in testo
        outputs[0], 
        skip_special_tokens=True
    )

    risultato = risposta.split(                # Estrae la parte successiva a "### Response:"
        "### Response:", 
        1
    )[1].strip()

    p_list.append(risultato)

    q_list.append(response)

    print(f"Added {risultato} in p_list and {response} in q_list")


In [ ]:
mauve_score = mauve.compute_mauve(
    p_text=p_list,
    q_text=q_list,
    device_id=0 if torch.cuda.is_available() else -1,  # usa GPU se disponibile
    max_text_length=1024,
)

print(f"MAUVE Score: {mauve_score.mauve}")